# Retrieve ach_id

In [2]:
import pandas as pd

df = pd.read_pickle('../data/processed_data/ccle_model.pkl')
df[(df['cid'].str.contains('cid'))&(df['tcga abbrev'] != 'NA')]

,depmap_id,ccle_name,OncotreePrimaryDisease,OncotreeLineage,tcga abbrev,cid
0,ACH-000001,NIHOVCAR3_OVARY,Ovarian Epithelial Tumor,Ovary/Fallopian Tube,TCGA_OV,cid_131
7,ACH-000008,A101D_SKIN,Melanoma,Skin,TCGA_SKCM,cid_5
17,ACH-000019,MCF7_BREAST,Invasive Breast Carcinoma,Breast,TCGA_BRCA,cid_86
26,ACH-000028,KPL1_BREAST,Invasive Breast Carcinoma,Breast,TCGA_BRCA,cid_76
33,ACH-000035,NCIH1650_LUNG,Non-Small Cell Lung Cancer,Lung,TCGA_LUSC,cid_122
...,...,...,...,...,...,...
1111,ACH-001190,SKMEL2_SKIN,Melanoma,Skin,TCGA_SKCM,cid_149
1115,ACH-001196,SMSCTR_SOFT_TISSUE,Rhabdomyosarcoma,Soft Tissue,TCGA_SARC,cid_155
1198,ACH-001374,PA1_OVARY,Ovarian Germ Cell Tumor,Ovary/Fallopian Tube,TCGA_OV,cid_132
1237,ACH-001418,UWB1289_OVARY,Ovarian Epithelial Tumor,Ovary/Fallopian Tube,TCGA_OV,cid_182


In [17]:
ach_list = sorted(df[(df['cid'].str.contains('cid'))&(df['tcga abbrev'] != 'NA')]['depmap_id'].unique())
len(ach_list)

75

# Download cell line-specific GRN

In [ ]:
import wget
for ach_id in ach_list:
    url = 'https://granddb.s3.us-east-2.amazonaws.com/cells/networks/ccle/'+ach_id+'.csv'
    wget.download(url)

# merge by mean

average differentially targeting TF per gene

In [3]:
import os
import glob
import tqdm

gene_list = []
tf_list = []
for f in tqdm.tqdm(glob.glob('../data/required_files/GRAND/*.csv')):
    ach_id = os.path.basename(f).split('.csv')[0]
    df = pd.read_csv(f, header=0, index_col=0) # TFs by targeted genes
    # averaging across targetting TFs for every gene
    gene_df = df.mean(axis=0).to_frame(name=ach_id) # rows are genes
    # averaging across targetting Genes for every TF
    tf_df = df.mean(axis=1).to_frame(name=ach_id) # rows are TFs
    # append to list
    gene_list.append(gene_df)
    tf_list.append(tf_df)

gene_df = pd.concat(gene_list, axis=1) # gene by ach_id
tf_df = pd.concat(tf_list, axis=1) # TF by ach_id

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 92/92 [20:32<00:00, 13.40s/it]


In [4]:
gene_df.head()

,ACH-000169,ACH-001196,ACH-000046,ACH-000338,ACH-000614,ACH-000288,ACH-000617,ACH-000348,ACH-000424,ACH-000421,...,ACH-001374,ACH-000713,ACH-000552,ACH-001751,ACH-000148,ACH-000001,ACH-000572,ACH-000861,ACH-000644,ACH-000367
A1BG,-0.299666,-0.011162,-0.214251,-0.276830,-0.212126,-0.231345,-0.188854,-0.207681,-0.169406,-0.202404,...,-0.170087,-0.265006,-0.538154,-0.236952,-0.178683,-0.525236,-0.199666,-0.095337,-0.269787,-0.313922
A1CF,-0.201265,0.729596,0.101080,0.143558,0.149901,0.358052,0.117603,0.068618,0.298880,0.107398,...,0.092415,0.215719,0.051914,0.198146,0.109620,0.343744,0.120323,0.082402,0.177089,0.039686
A2M,0.497679,0.652650,0.201999,0.473142,0.185194,0.100297,0.385606,0.238121,0.155217,0.116464,...,0.228945,0.416544,-0.211909,-0.051209,0.047856,-0.056941,0.123359,0.179650,0.190894,-0.473977
A2ML1,0.007306,-0.106903,-0.087535,-0.131794,0.006409,-0.094967,-0.063334,-0.049637,-0.049140,0.476519,...,-0.056919,-0.011083,0.669009,-0.059566,-0.131539,-0.154042,-0.082585,-0.112148,-0.038790,-0.187280
A3GALT2,-0.053190,0.015729,-0.153231,-0.019116,0.119296,-0.019289,-0.198657,-0.025494,-0.143006,-0.168078,...,-0.149175,-0.048263,-0.302745,-0.332015,-0.200452,0.282573,-0.145462,-0.191439,-0.305439,-0.245465


In [5]:
tf_df.head()

,ACH-000169,ACH-001196,ACH-000046,ACH-000338,ACH-000614,ACH-000288,ACH-000617,ACH-000348,ACH-000424,ACH-000421,...,ACH-001374,ACH-000713,ACH-000552,ACH-001751,ACH-000148,ACH-000001,ACH-000572,ACH-000861,ACH-000644,ACH-000367
Row,,,,,,,,,,,,,,,,,,,,,
AHR,-0.500804,-0.487930,-0.458893,-0.447264,-0.450421,-0.529475,-0.465225,-0.490830,-0.496814,-0.459896,...,-0.467665,-0.459262,-0.506618,-0.434746,-0.520891,-0.559471,-0.517853,-0.502909,-0.500713,-0.593922
AHRR,-0.386465,-0.375960,-0.345143,-0.330285,-0.331325,-0.416058,-0.351429,-0.375283,-0.386713,-0.345086,...,-0.352252,-0.346579,-0.389054,-0.319875,-0.407931,-0.452768,-0.406074,-0.389802,-0.386500,-0.483625
AIRE,-0.576853,-0.527831,-0.544948,-0.541137,-0.564844,-0.525771,-0.544952,-0.557142,-0.495561,-0.553799,...,-0.510033,-0.551508,-0.564288,-0.556821,-0.547418,-0.550612,-0.476119,-0.542965,-0.518622,-0.562070
ALX1,-0.711667,-0.683453,-0.702075,-0.653949,-0.705683,-0.657001,-0.672026,-0.681893,-0.617096,-0.676427,...,-0.656965,-0.667700,-0.712556,-0.706293,-0.641557,-0.636835,-0.654973,-0.712162,-0.713081,-0.712594
ALX3,-0.961348,-0.897073,-0.894902,-0.861220,-0.917905,-0.905810,-0.882226,-0.910829,-0.891809,-0.909342,...,-0.876834,-0.899320,-0.927218,-0.863308,-0.899991,-0.907540,-0.869493,-0.913935,-0.927833,-0.896697


In [7]:
# save to file
gene_df.T.to_csv('../data/required_files/GRAND.CCLE.Sample-Specific.meanDifferentialTargetingGene.csv', header=True, index=True, sep=",")
tf_df.T.to_csv('../data/required_files/GRAND.CCLE.Sample-Specific.meanDifferentialTargetingTF.csv', header=True, index=True, sep=",")

In [19]:
import pandas as pd

df = pd.read_csv('../data/required_files/GRAND.CCLE.Sample-Specific.meanDifferentialTargetingTF.csv', header=0, index_col=0)
df

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
ACH-000169,-0.299666,-0.201265,0.497679,0.007306,-0.053190,-0.170584,0.090679,-0.234531,-0.123293,-0.136407,...,0.208879,-0.538284,0.126268,0.255690,0.002113,0.178789,-0.296486,-0.279578,-0.222314,0.406294
ACH-001196,-0.011162,0.729596,0.652650,-0.106903,0.015729,-0.447029,0.224430,-0.072110,-0.395247,-0.306368,...,-0.160274,-0.025959,-0.120463,0.015061,-0.151726,0.505445,-0.502196,-0.032380,-0.343660,-0.069404
ACH-000046,-0.214251,0.101080,0.201999,-0.087535,-0.153231,-0.252386,0.110783,-0.167604,-0.252097,-0.183104,...,0.020412,-0.071551,0.223017,0.041794,-0.167613,0.011423,-0.253119,-0.122201,-0.235209,-0.130759
ACH-000338,-0.276830,0.143558,0.473142,-0.131794,-0.019116,-0.203104,0.164177,-0.039276,-0.308615,-0.272459,...,-0.134606,-0.108938,0.335616,0.068247,0.052472,0.046646,-0.069337,-0.170064,-0.185800,-0.216056
ACH-000614,-0.212126,0.149901,0.185194,0.006409,0.119296,-0.344985,0.005614,-0.080063,-0.226088,0.020420,...,0.059132,-0.032715,0.145653,0.135772,-0.092230,0.338914,-0.408243,-0.210199,-0.105054,-0.063571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-000001,-0.525236,0.343744,-0.056941,-0.154042,0.282573,-0.351507,-0.029927,-0.431426,-0.503158,-0.513332,...,-0.211249,0.105525,0.284483,0.273220,-0.342492,-0.472938,0.037641,0.197513,-0.053483,-0.133653
ACH-000572,-0.199666,0.120323,0.123359,-0.082585,-0.145462,-0.137441,0.115776,-0.216757,-0.261262,-0.129779,...,-0.066009,0.016034,0.151216,0.084230,-0.155969,-0.000506,-0.247723,-0.214792,-0.197560,-0.134230
ACH-000861,-0.095337,0.082402,0.179650,-0.112148,-0.191439,-0.232632,0.085148,-0.231885,-0.254698,-0.268872,...,0.024567,-0.095617,0.238777,0.265722,-0.178848,0.056981,-0.305983,-0.068936,-0.204158,-0.120230
ACH-000644,-0.269787,0.177089,0.190894,-0.038790,-0.305439,-0.254890,-0.007364,-0.257146,-0.283937,-0.235417,...,0.059710,0.105333,0.147567,0.137301,-0.104522,0.168186,-0.271527,-0.142541,-0.154583,-0.207443


# Binarize probability

"The edges weights are computed by PANDA, PUMA, OTTER, and LIONESS algorithms. They usually vary between -20 and 20. A large value means a high likelihood of the existence of an edge between two nodes, a low negative value means a small probability of interaction between two given nodes. (ref: https://grand.networkmedicine.org/help/)"

In [20]:
cate_df = df.applymap(lambda x: 1 if x>0 else 0)
cate_df

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
ACH-000169,0,0,1,1,0,0,1,0,0,0,...,1,0,1,1,1,1,0,0,0,1
ACH-001196,0,1,1,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0
ACH-000046,0,1,1,0,0,0,1,0,0,0,...,1,0,1,1,0,1,0,0,0,0
ACH-000338,0,1,1,0,0,0,1,0,0,0,...,0,0,1,1,1,1,0,0,0,0
ACH-000614,0,1,1,1,1,0,1,0,0,1,...,1,0,1,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-000001,0,1,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,1,1,0,0
ACH-000572,0,1,1,0,0,0,1,0,0,0,...,0,1,1,1,0,0,0,0,0,0
ACH-000861,0,1,1,0,0,0,1,0,0,0,...,1,0,1,1,0,1,0,0,0,0
ACH-000644,0,1,1,0,0,0,0,0,0,0,...,1,1,1,1,0,1,0,0,0,0
